In [3]:
from Crypto.Hash import keccak

def keccak256(string: str) -> str:
    new_cal = keccak.new(digest_bits=256)
    return new_cal.update(string.encode()).hexdigest()

keccak256('123')

'64e604787cbf194841e7b68d7cd28786f6c9a0a3ab9f8b0a0e87cb4387ab0107'

In [16]:
import json
from mimc import Zero, FieldNumber as FN, MiMCSponge as MiMC

series_dct = json.load(open('./dct-serialize.json'))

assert bin(len(series_dct) + 1)[2:].count('1') == 1, "Length doesn't match"
assert [l[0] for l in series_dct] == list(range(len(series_dct))), "Index doesn't match"